In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# DirPath = ('/content/drive/My Drive/')

In [2]:

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
#current version of Accelerate on GitHub breaks QLoRa
#Using standard pip instead
!pip install -q -U accelerate
!pip install -q -U datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
beatrix-jupyterlab 2023.621.222118 requires jupyter-server~=1.16, but you have jupyter-server 2.6.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cudf 23.8.0 requires pyarrow==11.*, but you have pyarrow 16.1.0 which is incompatible.
cuml 23.8.0 requires da

In [3]:
!pip freeze > kaggle_requirements.txt


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSequenceClassification, IntervalStrategy, BloomForSequenceClassification, EarlyStoppingCallback

In [5]:
model_name = "bigscience/bloom-560m"#"bigscience/bloom-3b"#"bigscience/bloom-560m"

#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [6]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
# import wandb

# wandb.login(key="e0b5e8662a12704a23b8b8dc633d1d688941df16")

import os
os.environ["WANDB_DISABLED"] = "true"

In [8]:
# model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map={"":0})
model = BloomForSequenceClassification.from_pretrained(model_name, quantization_config=quant_config, device_map={"":0},num_labels=2)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model.gradient_checkpointing_enable()

In [10]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"#"CAUSAL_LM"
)

model = get_peft_model(model, config)

In [11]:
# from datasets import load_dataset
# data = load_dataset("Abirate/english_quotes")
# data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
fake_df = pd.read_csv("/kaggle/input/banfakenews/Fake-1K.csv")[ ['headline', 'content', 'label'] ]
authentic_df = pd.read_csv("/kaggle/input/banfakenews/Authentic-48K.csv")[ ['headline', 'content', 'label'] ]

# Rename columns
fake_df = fake_df.rename(columns={'headline': 'Headline', 'content': 'Content', 'label': 'Label'})
authentic_df = authentic_df.rename(columns={'headline': 'Headline', 'content': 'Content', 'label': 'Label'})

# Split fake data into train and test sets
fake_train, fake_test = train_test_split(fake_df, test_size=0.2, random_state=42)

# Split authentic data into train and test sets
authentic_train, authentic_test = train_test_split(authentic_df, test_size=0.2, random_state=42)

# Concatenate the train and test sets for both fake and authentic data
train_df = pd.concat([fake_train, authentic_train])
val_df = pd.concat([fake_test, authentic_test])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
# from sklearn.model_selection import train_test_split
HCSep = " \\\\ "
def pandaToList(dataset):
    text_list = []
    label_list = []
    for i in range(len(dataset['Headline'])):
      text_list.append(dataset['Headline'].iloc[i]+HCSep+dataset['Content'].iloc[i])
      label_list.append(int(dataset['Label'].iloc[i]))
    return text_list, label_list

train_texts, train_labels =  pandaToList(train_df)
val_texts, val_labels = pandaToList(val_df)

In [14]:
max_length=512
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length)

In [15]:
class ViolenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ViolenceDataset(train_encodings, train_labels)
val_dataset = ViolenceDataset(val_encodings, val_labels)

In [16]:
# print(train_dataset[:2])
# model(input_ids=train_dataset[:1]['input_ids'],attention_mask=train_dataset[:1]['attention_mask'])

In [17]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    print("\nAfter iteration multiple of 2514\n")
    print({
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1": f1
    })
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [18]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    # train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        save_total_limit = 1,
        warmup_steps=1,
        # max_steps=20,
        num_train_epochs=2,
        learning_rate=2e-5,
        fp16=True,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        metric_for_best_model = 'f1',
        eval_strategy = "epoch",
        save_strategy = "epoch",
        weight_decay=0.005,
        eval_steps = 1,
        logging_steps=1,
        save_steps = 1,
        load_best_model_at_end=True,
        report_to="none"
    ),
    # data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.000000,0.043374,0.995898,0.975641,0.941743,0.958030
1,0.000000,0.033302,0.996499,0.985856,0.943923,0.963891



After iteration multiple of 2514

{'accuracy': 0.9958983593437375, 'precision': 0.9756407502483033, 'recall': 0.9417427785854244, 'f1': 0.9580300688449175}


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



After iteration multiple of 2514

{'accuracy': 0.9964985994397759, 'precision': 0.9858559641797704, 'recall': 0.9439226344731686, 'f1': 0.9638910663261436}


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=9994, training_loss=0.1572928100538082, metrics={'train_runtime': 27726.397, 'train_samples_per_second': 2.884, 'train_steps_per_second': 0.36, 'total_flos': 7.4446144536576e+16, 'train_loss': 0.1572928100538082, 'epoch': 1.999749881193567})

In [19]:
import os

# Specify the directory where you want to save the model
output_dir = "./path/to/save/directory"

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save the trained model
trainer.save_model(output_dir)


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
from sklearn.metrics import classification_report

test_df = pd.read_csv('/kaggle/input/newnewssep2023/new_fake_real_merged.csv')[ ['Headline','Content','Label']]
test_texts, test_labels = pandaToList(test_df)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)
test_dataset = ViolenceDataset(test_encodings, test_labels)

predictions = trainer.predict(test_dataset)

# Get predicted labels and true labels
predicted_labels = predictions.predictions.argmax(axis=1)
true_labels = predictions.label_ids

# Generate the classification report
class_report = classification_report(true_labels, predicted_labels, digits=5)

print(class_report)


After iteration multiple of 2514

{'accuracy': 0.3565217391304348, 'precision': 0.25276067624352583, 'recall': 0.3565217391304348, 'f1': 0.2810944091573873}
              precision    recall  f1-score   support

           0    0.09259   0.03261   0.04823       460
           1    0.41293   0.68043   0.51396       460

    accuracy                        0.35652       920
   macro avg    0.25276   0.35652   0.28109       920
weighted avg    0.25276   0.35652   0.28109       920



In [21]:
# text = "Ask not what your country"
# device = "cuda:0"
# inputs = tokenizer(text, return_tensors="pt").to(device)

# print(inputs)

# # outputs = model.generate(**inputs, max_new_tokens=20)
# outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))